# Simulating structurally variable Nuclear Pore Complexes for SMLM 
This Jupyter-Lab contains all relevant information to simulate variable NPCs and export them as CSVs

## General notes for not-yet programmers: 
This notebook is mainly written in python. It does load or export things that are not python, though: 
a config.yaml file contains standard parameters that are translated to python, and may be modified using python syntax. 
"null" in the .yaml file does not mean 0, but is equivalent to python "None": this value will be ignored in this framework. 

### Commenting/Uncommenting: 
A comment in python is, amongst other things, any content preceded by a "#". "Uncomment" means to remove the first "#"  

## Import relevant packages

In [ ]:
import exportCSV #exports data and metadata
import NPC 
import NPC_plotting
import Analyse_deformed
import json
import numpy as np
import csv

In [ ]:
%matplotlib widget 

## Specify directory in which to export data
Tip: It is good practice for the output directory to be distinct from the directory this jupyter-lab lives in 
Filepaths use backslashes "\\" on windows and forward slashes "/" on Mac or Linux 

In [ ]:
data_dir = './data/' # Directory for output files

## Select config file for simulation settings

In [ ]:
config = 'config.yaml'

The config file is now transformed into a python dictionary called "var" (Short for variables).  
Any entry in var can be edited and new entries can be added. var tells the simulator which properties the simulated NPCs should have.  
Entries in var are accessed or modified via a dictionary lookup with the following general notation:  
var["keyname"] = modified_value.  
Let's have a look at what's in var right now:  

In [ ]:
var = NPC.getVars(config) # Transform config file into a python dictionary 
print(json.dumps(var, indent = 4)) # Print var in an easily readable format 

# 1. Adjust simulation parameters

## Number of NPCs to be simulated
**NPCi** is the index out of NPC to be shown in any detail-plot or animation. The first NPC is indexed with 0, the last NPC with n-1.  
**var["seed"]** sets a random seed for reproducibility. Choose any number but 0. Simulated NPCs with the same parameters and same random seed will always look the same  
set **var["seed"] = False** to simulate different NPCs for every run

In [ ]:
var["n"] = 49 # Number of NPCs to be simulated
NPCi = 0 # index out of n of NPC to be shown in any detail-plot or animation. 0-indexed.  
var["seed"] = 51234 #seed for reproducibility. Any number but 0

# Select one or more nups, their terminus, and an NPC model for simulation 

## List of all available models 
Click on the cell below and then click on the blue vertical bar to the left of the cell to show/hide the table of model info

| Model                       | Substructure   | Origin                           | Resolution* $\overset{\circ}{A}$ |   |
|-----------------------------|----------------|----------------------------------|---------------------------|---|
| 5A9Q<sup>1</sup>    | ORs            | HeLa, isolated NE                | 23                        |   |
| 7PEQ<sup>2</sup>    | ORs            | DLD-1, native environment        | 35                        |   |
| 5IJN<sup>3</sup>    | IR             | HeLa, isolated NE                | 21.4                      |   |
| 5IJO<sup>3</sup>    | IR             | HeLa, isolated NE                | 21.4                      |   |
| 7PER<sup>2</sup>    | IR             | DLD-1, native environment        | 35                        |   |
| 7R5K<sup>4</sup> | Whole scaffold | HeLa, isolated NE                | 12                        |
| 7R5J<sup>4</sup> | Whole scaffold | HeLa, HEK293, native environment | 50                        |

*: As reported in PDB  
1: von Appen, Alexander et al. “In situ structural analysis of the human nuclear pore complex.” Nature vol. 526,7571 (2015): 140-143. doi:10.1038/nature15381  
2: Schuller, Anthony P et al. “The cellular environment shapes the nuclear pore complex architecture.” Nature vol. 598,7882 (2021): 667-671. doi:10.1038/s41586-021-03985-3  
3: Kosinski, Jan et al. “Molecular architecture of the inner ring scaffold of the human nuclear pore complex.” Science (New York, N.Y.) vol. 352,6283 (2016): 363-5. doi:10.1126/science.aaf0643  
4: Mosalaganti, Shyamal et al. “AI-based structure prediction empowers integrative structural analysis of human nuclear pores.” Science (New York, N.Y.) vol. 376,6598 (2022): eabm9506. doi:10.1126/science.abm9506  

## List of all selectable nups
Click on the cell below and then click on the blue vertical bar to the left of the cell to show/hide full table of nup info

| Model                                            | Nup             | N-term. | C-term.                            | n in CR | NR | IR | bridge | total |
|--------------------------------------------------|-----------------|---------|------------------------------------|---------|----|----|--------|-------|
| 5A9Q                                             | Nup43            | 4      | 380                                | 2       | 2  |    |        | 4     |
|                                                  | Nup160          | 41      | 1201                               | 2       | 2  |    |        | 4     |
|                                                  | Nup37           | 9       | 326                                | 2       | 2  |    |        | 4     |
|                                                  | Sec13           | 14      | 304                                | 2       | 2  |    |        | 4     |
|                                                  | Seh1            | 1       | 322                                | 2       | 2  |    |        | 4     |
|                                                  | Nup85           | 9       | 475                                | 2       | 2  |    |        | 4     |
|                                                  | Nup155          | 20      | 730                                |         |    |    | 2      | 2     |
|                                                  | Nup133          | 518     | 1156                               | 2       | 2  |    |        | 4     |
|                                                  | Nup96           | 277     | 751                                | 2       | 2  |    |        | 4     |
|                                                  | Nup107          | 150     | 924                                | 2       | 2  |    |        | 4     |
| 7PEQ                                             | Nup133          | 518     | 1156                               | 2       | 2  |    |        | 4     |
|                                                  | Nup107          | 150     | 924                                | 2       | 2  |    |        | 4     |
|                                                  | Nup96           | 333     | 922                                | 2       | 2  |    |        | 4     |
|                                                  | Sec13           | 14      | 304                                | 2       | 2  |    |        | 4     |
|                                                  | Seh1            | 5       | 320                                | 2       | 2  |    |        | 4     |
|                                                  | Nup85           | 20      | 651                                | 2       | 2  |    |        | 4     |
|                                                  | Nup43           | 4       | 380                                | 2       | 2  |    |        | 4     |
|                                                  | Nup160          | 78      | 1195                               | 2       | 2  |    |        | 4     |
|                                                  | Nup37           | 18      | 324                                | 2       | 2  |    |        | 4     |
| 7PER                                             | Nup205          | 9       | 1692                               |         |    | 4  |        | 4     |
|                                                  | Nup155          | 20      | 1375                               |         |    | 4  |        | 4     |
|                                                  | Nup93           | 173     | 815                                |         |    | 4  |        | 4     |
|                                                  | Nup54           | 128     | 493                                |         |    | 4  |        | 4     |
|                                                  | Nup58<sup>1</sup>| 248    | 418                                |         |    | 4  |        | 4     |
|                                                  | Nup62           | 334     | 502                                |         |    | 4  |        | 4     |
| 5IJN                                             | Nup205          | 9       | 1692                               |         |    | 4  |        | 4     |
|                                                  | Nup54           | 128     | 493                                |         |    | 4  |        | 4     |
|                                                  | Nup58           | 248     | 418                                |         |    | 4  |        | 4     |
|                                                  | Nup155          | 20      |  2 x 863                           |         |    | 4  |     2  | 6     |
|                                                  |                 |         | 4 x 1375                           |         |    |    |        |       | 
|                                                  | Nup93           | 1       | 815                                |         |    | 4  |        | 4     |
|                                                  | Nup62           | 334     | 502                                |         |    | 4  |        | 4     |
| 5IJO                                             | Nup54           | 128     | 493                                |         |    | 4  |        | 4     |
|                                                  | Nup188          | 1       | 1564                               |         |    | 2  |        | 2     |
|                                                  | Nup205          | 9       | 1692                               |         |    | 2  |        | 2     |
|                                                  | Nup155          | 20      |  2 x 863                           |         |    | 4  |     2  | 6     |
|                                                  |                 |         | 4 x 1375                           |         |    |    |        |       | 
|                                                  | Nup93           | 1       | 815                                |         |    | 4  |        | 4     |
|                                                  | Nup58<sup>1</sup>  | 248     | 418                                |         |    | 4  |        | 4     |
|                                                  | Nup62           | 334     | 502                                |         |    | 4  |        | 4     |
| 7R5J                                             |
|                                                  | Nup210<sup>2</sup> | 2       | 1832                               |         |    | 8  |        | 8     |
|                                                  | ALADIN          | 2       | 486                                |         |    | 2  |        | 2     |
|                                                  | Nup93           | 2       | 819                                |         |    | 4  |        | 4     |
|                                                  | Nup93           | 94      | 819                                | 2       | 1  |    |        | 3     |
|                                                  | Nup188          | 2       | 1749                               |         |    | 2  |        | 2     |
|                                                  | Nup205          | 2       | 2012                               | 2       | 1  | 2  |        | 5     |
|                                                  | Nup155          | 10      | 1391                               |         |    | 4  | 2      | 6     |
|                                                  | NDL1            | 2       | 674                                |         |    | 2  |        | 2     |
|                                                  | Nup35           | 86      | 326                                |         |    | 4  |        | 4     |
|                                                  | Nup54           | 111     | 493                                |         |    | 4  |        | 4     |
|                                                  | Nup58<sup>1</sup> | 246     | 418                                |         |    | 4  |        | 4     |
|                                                  | Nup62           | 332     | 502                                | 1       |    | 4  |        | 5     |
|                                                  | Nup133          | 71      | 1156                               | 2       | 2  |    |        | 4     |
|                                                  | Nup107          | 144     | 925                                | 2       | 2  |    |        | 4     |
|                                                  | Nup96           | 232     | 937                                | 2       | 2  |    |        | 4     |
|                                                  | SEC13           | 2       | 302                                | 2       | 2  |    |        | 4     |
|                                                  | SEH1            | 2       | 324                                | 2       | 2  |    |        | 4     |
|                                                  | Nup85           | 2       | 656                                | 2       | 2  |    |        | 4     |
|                                                  | Nup43           | 2       | 380                                | 2       | 2  |    |        | 4     |
|                                                  | Nup160          | 38      | 1436                               | 2       | 2  |    |        | 4     |
|                                                  | Nup37           | 5       | 326                                | 2       | 2  |    |        | 4     |
|                                                  | ELYS            | 2       | 1005                               |         | 2  |    |        | 2     |
|                                                  | Nup98           | 731     | 880                                |         | 1  |    |        | 1     |
|                                                  | Nup98           | 597     | 615                                | 2       |    | 4  |        | 6     |
|                                                  | Nup214          | 700     | 972                                | 1       |    |    |        | 1     |
|                                                  | Nup88           | 7       | 741                                | 1       |    |    |        | 1     |

1: Reported as p58/p45  
2: Primarily in Nuclear Envelope lumen



You can write one nup **var["nup"]**, e.g. nup107, either as ("nup107",) or as "nup107" --- capitalisation doesn't matter.   
protein termini are listed equivalently.  
Multiple nups must be listed as a tuple, for instance ("nup107", "nup96"), and protein termini must have the same order as their corresponding Nup.  
**var["nup"]** = ("nup107", "nup96") and  
**var["term"]** = ("C", "N")  
**var["model"]**  = "7r5j" for instance means:   
"nup107-C, and N-nup96 as presented in PDB model '7r5j' are selected as virtual sites to attach a label, for instance a fluorophore"  
**var["rel"]**: relevant for multicolour simulations.  
If true, it selects the first listed nup as a reference to make it comparable to single-channel simulations of this nup. 
default is **false**  

In [ ]:
var["nup"] = ("nup107",)
var["term"] = ("C",)
var["model"] = "7r5J"

# 2. Simulate variability 
## 2.1. Irregular variability 
**mag**: magnitude of deforming forces in x-y direction; the higher, the more deformed. recommended 0 - 20  
**zmag**: magnitude of deforming offsets in z direction; the higher, the more deformed. default mag/2. 

In [ ]:
# # Variability parameters
var["mag"] = 20 # Magnitude of irregular variability. 0: No deformation, 15: Strong deformation
var["zmag"] = var["mag"]/2 # magnitude of offset in z. Not computed via springs 

### Irregular variability - advanced settings
Following parameters can be modified to change behaviour of deformations, however this is not recommended.  
**sigmamult**: controls correlation of forces, higher values result in more correlated forces, thus smoother deformations. default = 0.5.  
**nConnect**: Adjusts number of springs. Default: 2, i.e each node is connected to the two nearest neighbours in clockwise direction, and to the 2 nearest neighbours in anticlockwise direction.  
Uncomment the cell below to try this out. 

In [ ]:
#var["sigmamult"] = 0.5
#var["nConnect"] = 2
#var["tlast"] = 20

## 2.2. Geometric variability
Keys are indicated in bold lettering. Variables can for instance be modified via var["rnew"] = new_value. 
### Radius variability  
- **rnew** new mean radius
- **rsigma** standard deviation of radius  
### Variability of twist angle between Nucleoplasmic and Cytoplasmic side  
- **thetanew** new twist angle
- **thetasigma** standard deviation of twist angle
### Variability of distance between Nucleoplasmic and Cyotplasmic side  
- **dnew** new mean distance
- **dsigma** standard deviation of distance 
### Variability of ellipticity
- **elliptnew**  new mean input ellipticity as ratio of minor to major axis
- **elliptsigma**  standard deviation of ellipticity

### Others
**symmet**: Rotational symmetry of the NPC, default **8**  
**kappa**: Random independent tilt of Nucleoplasmic and Cytoplasmic side. Default **null** means no tilt.  
kappa=0 is random uniform tilt. The higher kappa, the less tilt from the default position is applied.  
**shiftsigma**: x and y offset of either Nucleoplasmic and Cytoplasmic side are independently sampled from a normal distribition with sigma = shiftsigma [nm]


## Modify geometric variability 
EXAMPLE: Generate NPCs with radius varying by a standard deviation of 5 nm that are also have a slight tilt in their nucleoplasmic and cytoplasmic side 

In [ ]:
var["rsigma"] = 5
var["kappa"] = 40

# 3. Run simulations
NPC.getNPCs(var) simulates NPCs that vary as described in var.  
The output additionally contains metadata.  

WARNING: Tilts (see var["kappa"]) and shifts (see var["shiftsigma"]) of the Nucleoplasmic and Cytoplasmic side are not yet applied to data contained in NPCs.  
This will be done in a later step.  

## A note on time
Irregular shapes are computed by applying forces to a damped spring system, which will thus deform to a new equilibrium shape.  
NPCs therefore contains temporal information stored as timesteps: The first timestep is the model before deformation,  
the last step is the model with a new equilibrium shape. As super-resolution microscopy techniques typically lack the temporal resolution   
to access NPC dynamics, we are only interested in the model at the last timestep.  
The last timestep **var["tlast"]** is 20 by default, sufficient to accomodate deformations with typical settings. 
Higher values might increase compute time, lower values might not offer enough time for the model to equilibrate.  
However if a large number of NPCs (n >> 5000) are computed at low deformation magnitudes (var["mag"] < 5), it might be beneficial to reduce **var["tlast"]**. 
Further below we introduce visualisation functions to check whether NPCs were able to equilibrate. 

In [ ]:
#var["tlast"] = 20
NPCs = NPC.getNPCs(var) # Dictionary of all simulated NPCs

In [ ]:
#print(NPCs.keys()) 

## Extract coordinates from variable NPCs. 
NPC.getNPCcoords() extracts coordinates of the final deformed state of the model. var["kappa"] and var["shiftsigma"] is applied here if applicable. 
rows of NPCscoords are x, y, z, nup index, NPC index.  
nup index specifies the nup: Always 0 in single-channel simulations  
NPC index specifies the NPC out of n

In [ ]:
NPCscoords = NPC.getNPCcoords(NPCs, var) 

In [ ]:
print(NPCscoords)

## Arrange NPCs on a grid
NPCscoords contains relative coordinates, i.e. NPCs are more or less 0-centred. They would overlap if plotted. 
OffsetNPCs() arranges these NPCs on a grid using a given offset

In [ ]:
offset = 1.5 * np.mean(NPCs["rexp"]) # set offset to 1.5 * the expected NPC radius 
OffsetNPCs = NPC_plotting.OffsetNPCs(NPCscoords, offset)

# 4. Visualise NPCs
## Overview of all deformed NPCs

In [ ]:
NPC_plotting.plotOverview(OffsetNPCs, NPCs, var, width = 9)

## Plot position of each nup over time for a given NPC
This allows to see whether all proteins of an NPC with index NPCi have reached a new equilibrium

In [ ]:
NPC_plotting.positionVStime(NPCs, index = NPCi, legend = True)

## Explore a specific NPC in greater detail 
Explore a specific NPC with index NPCi in greater detail, it can be plotted as a 2D or 3D plot.
3D plots are interactive - click and drag to view it from all angles. 

In [ ]:
# Tip: replace NPCi with any integer between 0 and n-1 to view a different NPC
NPC_plotting.plotDetail(NPCscoords, NPCs, var, index = NPCi, width = 5, mode = "3D")

# Generate an animation of deformation 
Replace **name = None** with name = "examplename" save the animation file in the data directory.  
The name needs to be surrounded by quotation marks. Avoid special characters or whitespaces. 
## View deformation of an NPC with index NPCi in detail 

In [ ]:
#name = None #
#NPC_plotting.AnimateDetail(NPCs, var, index = NPCi, width = 5, directory = data_dir, name = name, ext = ".gif")

## View deformations of all NPCs 
This is a great way to quickly check all NPCs have reached a new equlibrium. 
If this is not the case (unlikely for most standard parameters), the last timestep var["tlast"] can be adjusted before computing deformations (see further above). 

In [ ]:
#name = None # see above
#NPC_plotting.AnimateOverview(NPCs, OffsetNPCs, var, width = 6, directory = data_dir, name = name, ext = ".gif")

# 4. Analyse generated NPCs 
Features will be explained in more details further down, where they are saved in a CSV file.  
Variable names starting with "ellipse" include features derived from fitting 3D ellipses to NPCs or parts thereof.  
Variable names starting with "circle" include features derived from fitting 3D circles to NPCs or parts thereof.   
Variable names that end with "allrings" mean that the circle or ellipses were fitted to each individual ring of the NPC. (See publication for definition of ring).  
Variable names that end with "CRNR" mean that circles or ellipses were fitted to each subcomplex,  
i.e. the cytoplasmic ring, nucleoplasmic ring, the bridge between outer ring and inner ring, and/or the inner ring.  

In [ ]:
ellipse_allrings = Analyse_deformed.Ellipses(NPCscoords, membership = NPCs["z_i_all"])
ellipse_CRNR = Analyse_deformed.Ellipses(NPCscoords, membership = NPCs["ringmemall"])

circle_allrings = Analyse_deformed.Circles(NPCscoords, membership = NPCs["z_i_all"])
circle_CRNR = Analyse_deformed.Circles(NPCscoords, membership = NPCs["ringmemall"])    

## Compute further features 
featuresAll: Per NPC: radius, residual sum of squares, residual sum of squares on circle-plane, residual sum of squares to circle-plane  
featuresElAll: Per NPC: Major axis ellipse, minor axis ellipse, minor/major axis ratio, rotational angle [rad], residual sum of squared distances of coordinates to the ellipse, residual sum of squared distances of coordinates projected onto the ellipse-plane from the ellipse, residual sum of squared distances of coordinates to the ellipse-plane.  
features3DAll: Each alternating by subcomplex: Centre (x,y,z), tilt-vector (x,y,z)/

In [ ]:
featuresAll, _, _ = Analyse_deformed.meanfeaturesC(NPCs, var, circle_allrings) # Circle features
featuresElAll, _, _ = Analyse_deformed.meanfeaturesE(NPCs, var, ellipse_allrings)
_, _, _, _, featuresel3DAll = exportCSV.col_features(NPCs, circle_CRNR, ellipse_CRNR)

# featuresAll: r, sqsum, residual, zsqsum of circle per NPC 
#featuresel3DAll: centre subcomplex, 1 centre subcomplex 2, centre subcomplex n, tilt subcomplex 1, tilt subcomplex 2 ...

## Histogram of features 
radius [nm]: Radius per NPC, determined by fitting circles to each NPC ring and averaging their radii  
minor/major ellip: Minor/major axis ratio per NPCs, determined by fitting an ellipse to each ring and averaging their values  
RSS ellipt. [nm]: Averaged per NPC: Residual sum of squared distances from ring-coordinates to ring-wise fitted ellipses   
tilt-dif. ellipt [rad]: Per NPC: tilt angle between Nucleoplasmic and Cytoplasmic side  
shift ellip [nm]: Distance of centres of Nucleoplasmic and cytoplasmic side in x-y dirction. Centres are determined by fitting an ellipse to each sub-complex  
Dist ellip [nm]: Distance of centres of Nucleoplasmic and cytoplasmic side in z dirction. Centres are determined by fitting an ellipse to each sub-complex  

In [ ]:
NPC_plotting.gethistdata(var, NPCs, featuresAll, featuresElAll, featuresel3DAll, width = 5, bins = None)

# 5. Export data

## Export NPC coordinates and simulation metadata
Note that here, non-offset NPCs are exported, assuming that any downstream simulation software will arrange NPCs on a grid. 
Offset NPCs can be exported instead, should this not be the case. 

In [ ]:
nameDict, name = exportCSV.names(var)
makecsv = exportCSV.MakeCSV(var, NPCs, NPCscoords, nameDict, name, data_dir) 

In [ ]:
exportCSV.previewCSV(makecsv.csvpath, lines = 5)

## Export NPC features

### Features of whole NPCs

In [ ]:
featurescsv = exportCSV.featuresCSV(NPCs, var, name, circle_allrings, ellipse_allrings, data_dir)

### Columns: 
**NPC**: index per NPC  
**c_r**: average radius per NPC  
**c_sse**:  Averaged per NPC: Residual sum of squared distances from ring-coordinates to ring-wise fitted circles   
**c_sselat**: Averaged per NPC: Residual sum of squared distances on the circle-plane from ring-coordinates to ring-wise fitted circles   
**c_sseax**: Averaged per NPC: Residual sum of squared distances from ring-coordinates to a ring-wise fitted circle-plane  
**c_x0**, **c_y0**, **c_z0**: Centre of NPC: centre-coordinates obtained by averaging centres of circles fitted to each NPC ring  
**c_tilt_x**, **c_tilt_y**, **c_tilt_z**: Average tilt of NPC: obtained by averaging the tilt of the circle-plane fitted to each NPC ring  

The following values are obtained by fitting an ellipse to each ring of the NPC and averaging ellipse features for each NPC  
**el_major**: Average major axis per NPC  
**el_minor**: Average minor axis per NPC  
**el_q**: ratio minor/major axis, average per NPC  
**el_rot**: Planar rotation of the fitted ellipses averaged by NPC [rad]  
**el_ssum**: Residual sum of squared distances from ring-coordinates to ring-wise fitted ellipses  
**el_ssumXY**: Averaged per NPC: Residual sum of squared distances on the ellipse-plane from ring-coordinates to ring-wise fitted ellipses  
**el_ssumZ**: Residual sum of squared distances from ring-coordinates to a ring-wise fitted ellipse-plane   
**ce_x0**, **ce_y0**, **cez0**: Centre of NPC: centre-coordinates obtained by averaging centres of ellipses fitted to each NPC ring  
**e_tilt_x**, **e_tilt_y**, **e_tilt_z**: Average tilt of NPC: obtained by averaging the tilt of the ellipse-plane fitted to each NPC ring  


In [ ]:
exportCSV.previewCSV(featurescsv.csvpath, lines = 2)

### Features per NPC ring

In [ ]:
featurescsvRing = exportCSV.featuresCSV_rings(NPCs, var, name, data_dir, circle_allrings, ellipse_allrings)

NPC: index per NPC  

Example: n107_56nm indicates that features are determined for a Nup107 ring with z-position 56nm. Other Nups and rings are annotated in the same way   

**r_n107_56nm**: radius of a circle fitted to Nup107 ring at z-position 56.   
**el_minor_n107_56nm**, **el_major_n107_56nm**, **el_q_n107_56nm**: Minor axis, major axis, and their ratio of a fitted ellipse   

**SSE_n107_56nm**: Residual sum of squared distances for a fitted circle  
**el_ssum_n107_56nm**: Equivalent from fitting an ellipse  

**SSE_l_n107_56nm**: Residual sum of squared distances of coordinates projected onto the circle-plane to a fitted circle  
**el_ssumXY_n107_56nm**: Equivalent from fitting an ellipse   

**SSE_a_n107_56nm**: Residual sum of squared distances of coordinates to the fitted circle plane  
**el_ssumZ_n107_56nm**: Equivalent from fitting an ellipse   

**Cx_n107_56nm**, **Cy_n107_56nm**, **Cz_n107_56nm**: Centre determined by fitting a circle   
**eCx_n107_56nm**, **eCy_n107_56nm**, **eCz_n107_56nm**: Equivalent from fitting an ellipse  

**Tx_n107_56nm**, **Ty_n107_56nm**, **Tz_n107_56nm**: Tilt determined by fitting a circle  
**eTx_n107_56nm**', **eTy_n107_56nm**, **eTz_n107_56nm**: Equivalent from fitting an ellipse   

**el_rot_n107_56nm**: Rotation of fitted ellipse 

In [ ]:
exportCSV.previewCSV(featurescsvRing.csvpath, lines = 0)

### Features per subcomplex

In [ ]:
featuresCSVsubcomplex = exportCSV.featuresCSV_subcomplex(NPCs, circle_CRNR, ellipse_CRNR, name, data_dir)

**NPC**: index per NPC  
Example: CR stands for cytoplasmic ring. NR stands for Nucleoplasmic ring  

**r_CR**: Radius determined by fitting a circle to the CR  
**el_minor_CR**, **el_major_CR**, **el_q_CR**:  Minor axis, major axis, and their ratio determined by fitting an ellipse  

**SSE_CR**: Residual sum of squared distances for a fitted circle  
**el_ssum_CR**: Equivalent from fitting an ellipse  

**SSE_l_CR**: Residual sum of squared distances of coordinates projected onto the circle-plane to a fitted circle  
**el_ssumXY_CR**: Equivalent from fitting an ellipse  

**SSE_a_CR**: Residual sum of squared distances of coordinates to the fitted circle plane  
**el_ssumZ_CR**: Equivalent from fitting an ellipse  

**Cx_CR**, **Cy_CR**, **Cz_CR**: centre coordinates of a fitted circle  
**el_Cx_CR**, **el_Cy_CR**, **el_Cz_CR**: centre coordinates of a fitted ellipse  

**Tx_CR**, **Ty_CR**, **Tz_CR**: tilt determined by fitting a circle  
**el_ Tx_CR**, **el_Ty_CR**, **el_ Tz_CR**:  tilt determined by fitting an ellipse   

In [ ]:
exportCSV.previewCSV(featuresCSVsubcomplex.csvpath, lines = 1)